In [ ]:
import numpy as np
import sys
from collections import defaultdict

In [ ]:
def print_grid(grid):
    for i in grid:
        for j in i:
            if j == 0:
                print("_", end="  ")
            else: 
                print(int(j), end="  ")
        print("\n")

In [ ]:
def initialize_grid(grid):
    #Initialize the grid
    initial_state = {14:3, 16:8, 17:5, 20:1, 22:2, 30:5, 32:7, 38:4, 42:1, 46:9, 54:5, 61:7, 62:3, 65:2, 67:1, 76:4, 80:9}
    for k,v in initial_state.items():
        row = k//9
        col = k%9
        grid[row, col] = v
    
    print("Grid:\n")
    print_grid(grid)
    return grid

In [ ]:
# state_tree = {}
# values_tree = defaultdict(dict)

In [ ]:
def find_new_domain(grid, cell_idx, value, old_domain):
    rowid = cell_idx//9
    colid = cell_idx%9
    
    # Remove the value from the same row/column candidates
    for i in range(0, 9):
        if value in old_domain[(rowid, i)]:
            old_domain[(rowid, i)].remove(value)
        if value in old_domain[(i, colid)]:
            old_domain[(i, colid)].remove(value)
    
    # Remove the value from the same subsquare
    block_pos_x = rowid//3
    block_pos_y = colid//3
    block_start_pos = {0:0, 1:3, 2:6, 3:27, 4:30, 5:33, 6:54, 7:57, 8:60}
    start_pos = block_start_pos[block_pos_x*3+block_pos_y]
    positions = []
    for i in range(0,3):
        positions.append(start_pos+i)
        positions.append(start_pos+i+9)
        positions.append(start_pos+i+18)
    
    for i in positions:
        if value in old_domain[(i//9, i%9)]:
            old_domain[(i//9, i%9)].remove(value)
    
    return old_domain

In [ ]:
def find_candidates_for_exploration(grid, domain):
    
#     candidates_with_values = []
    candidates = []
    
    for i in range(0, 81):
        rowid = i//9
        colid = i%9
        
        if grid[rowid, colid] == 0:
            if len(domain[(rowid, colid)]) > 0:
                candidates.append(i)

    return candidates

In [ ]:
def initialize_domain(grid):
    domain = {}
    for i in range(0, 81):
        rowid = i//9
        colid = i%9
        
        if grid[rowid, colid] == 0:
            domain[(rowid, colid)] = list(range(1, 10))
            
            for j in range(0, 9):
                if grid[(rowid, j)] in domain[(rowid, colid)]:
                    domain[(rowid, colid)].remove(grid[(rowid, j)])
                if grid[(j, colid)] in domain[(rowid, colid)]:
                    domain[(rowid, colid)].remove(grid[(j, colid)])
            
            block_pos_x = rowid//3
            block_pos_y = colid//3
            block_start_pos = {0:0, 1:3, 2:6, 3:27, 4:30, 5:33, 6:54, 7:57, 8:60}
            start_pos = block_start_pos[block_pos_x*3+block_pos_y]
            positions = []
            for i in range(0,3):
                try:
                    if grid[((start_pos+i)//9, (start_pos+i)%9)] in domain[(rowid, colid)]:
                        domain[(rowid, colid)].remove(grid[((start_pos+i)//9, (start_pos+i)%9)])
                    if grid[((start_pos+9+i)//9, (start_pos+9+i)%9)] in domain[(rowid, colid)]:
                        domain[(rowid, colid)].remove(grid[((start_pos+9+i)//9, (start_pos+9+i)%9)])
                    if grid[((start_pos+18+i)//9, (start_pos+18+i)%9)] in domain[(rowid, colid)]:
                        domain[(rowid, colid)].remove(grid[((start_pos+18+i)//9, (start_pos+18+i)%9)])
                except Exception as ex:
                    print(ex)
                    print(start_pos, i, start_pos+i//9, start_pos+9+i//9, start_pos+18+i//9)
        else:
            domain[(rowid, colid)] = [grid[rowid, colid]]
    return domain
                

In [ ]:
def DFS(grid, domain, stateidx=0, tsidx=0, level=0):
    candidate_cells = find_candidates_for_exploration(grid, domain)
    
    if len(candidate_cells) == 0:
        if np.sum(grid) == 405:
            print("Solved!")
            print_grid(grid)
            sys.exit(1)
        else:
            print("Backtracking!")
            return "bt"

    gl_candidate_values = {}
    for cc in candidate_cells:
        gl_candidate_values[cc] = domain[(cc//9, cc%9)]
    print("level, all cand values@level: " , level, gl_candidate_values)

    new_dom = domain.copy()
    grid_copy = grid.copy()
    for cc in candidate_cells:
        for cv in gl_candidate_values[cc][-1::-1]:
            tsidx = tsidx+1
            values_tree[stateidx][cv] = True    
            grid_copy[cc//9, cc%9] = cv
            new_domain_with_new_value = find_new_domain(grid_copy, cc, cv, new_dom)
            new_domain_with_new_value[(cc//9, cc%9)] = [cv]
            print("DFS ON level, cell, val: ", level, cc, cv)
            op = DFS(grid_copy, new_domain_with_new_value, stateidx+1, tsidx, level+1)


In [ ]:
grid = [list(range(9,0,-1)), [2,4,6,0,7,3,0,8,0], [3,0,1,9,2,8,7,4,0], [1,2,8,5,3,7,0,9,4], [6,3,4,8,9,2,1,0,7],
       [7,9,0,4,6,1,8,3,2], [5,1,0,2,8,6,4,7,3], [4,7,0,3,0,9,5,0,0], [8,6,3,7,0,5,2,0,0]]
# grid = [[8,0,0,1,0,0,0,7,0], [0,2,0,0,4,0,8,0,0],[0,6,0,7,0,0,0,0,0],[0,0,0,4,7,0,9,0,8],[2,4,0,0,8,0,0,0,0],[0,3,8,0,0,0,0,0,5],[0,8,0,6,0,4,1,0,0],[9,0,0,0,0,7,2,0,4],[0,0,5,8,1,0,0,0,6]]
print(grid)

In [ ]:
UNEXPLORED = 0  #Cell is available.


# grid = np.zeros((9, 9))
# grid = initialize_grid(grid)
grid = np.array(grid)
domain = initialize_domain(grid)

DFS(grid, domain)